In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing as preproc
from scipy import stats
import numpy as np

In [2]:
df=pd.read_parquet("diabetic_data_cleaned_pca_som_ae.parquet")

In [3]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,ae_1,ae_2,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10
0,Caucasian,Female,[10-20),1,1,7,3,NaN,NaN,59,...,10.852900,15.065039,31.019421,27.719538,25.533045,7.944871,20.922789,27.010193,11.216609,20.678316
1,AfricanAmerican,Female,[20-30),1,1,7,2,NaN,NaN,11,...,18.226522,17.166298,23.886705,33.430027,47.993740,8.140186,39.252544,27.722694,7.005354,28.845524
2,Caucasian,Male,[30-40),1,1,7,2,NaN,NaN,44,...,13.771294,19.349968,48.049038,31.504131,22.436737,6.229624,17.802269,19.045977,23.747458,44.441738
3,Caucasian,Male,[40-50),1,1,7,1,NaN,NaN,51,...,8.532896,23.532095,22.429426,19.974939,25.424990,18.631826,11.400923,9.645898,21.622543,35.477718
4,Caucasian,Male,[50-60),2,1,2,3,NaN,NaN,31,...,11.166486,17.973852,21.544680,26.190376,11.214865,20.167032,30.240730,11.272454,58.370567,21.489260


In [4]:
df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'metformin', 'repaglinide', 'glimepiride',
       'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin',
       'glyburide-metformin', 'change', 'diabetesMed', 'readmitted', 'PC1',
       'PC2', 'PC3', 'som_categoricals_jaccard_0',
       'som_categoricals_jaccard_1', 'som_jaccard_0', 'som_jaccard_1',
       'som_euc_0', 'som_euc_1', 'ae_0', 'ae_1', 'ae_2', 'ae_3', 'ae_4',
       'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9', 'ae_10'],
      dtype='object')

In [5]:
DB_COLUMNS = [
  'PC1','PC2', 'PC3','ae_0', 'ae_1', 'ae_2', 'ae_3', 'ae_4',
       'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9', 'ae_10'
]

In [6]:
df[DB_COLUMNS]

,PC1,PC2,PC3,ae_0,ae_1,ae_2,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10
0,-0.214843,0.070744,-0.194516,22.876373,10.852900,15.065039,31.019421,27.719538,25.533045,7.944871,20.922789,27.010193,11.216609,20.678316
1,0.417941,-0.522845,0.160733,13.269567,18.226522,17.166298,23.886705,33.430027,47.993740,8.140186,39.252544,27.722694,7.005354,28.845524
2,-0.129162,-0.131116,-0.050765,21.936855,13.771294,19.349968,48.049038,31.504131,22.436737,6.229624,17.802269,19.045977,23.747458,44.441738
3,-0.333245,-0.143083,-0.009183,23.144642,8.532896,23.532095,22.429426,19.974939,25.424990,18.631826,11.400923,9.645898,21.622543,35.477718
4,0.640729,-0.446888,-0.059434,15.655556,11.166486,17.973852,21.544680,26.190376,11.214865,20.167032,30.240730,11.272454,58.370567,21.489260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100239,-0.226515,0.049629,-0.151316,20.304390,37.700432,38.159042,36.194859,32.095642,40.747505,31.212931,5.257920,19.935497,9.728429,28.007534
100240,0.266458,-0.086568,-0.008378,17.969046,32.420887,29.241774,33.622025,35.379620,26.705563,8.355609,30.576368,25.128113,21.916023,19.272825
100241,-0.277035,-0.027778,-0.406056,29.677853,29.270254,19.270786,29.298632,31.630203,47.947598,25.455599,1.833044,20.393873,18.221397,24.220598
100242,0.291241,0.333055,0.077571,38.435413,17.450762,30.016045,16.249697,25.172110,21.008617,15.578630,22.754118,47.332405,23.976475,21.576572


In [7]:
X=df[DB_COLUMNS]

In [8]:
# Scale the independent variables
#scaler = StandardScaler()
X_scaled = preproc.MinMaxScaler().fit_transform(X)

In [9]:
# Apply DBSCAN
epsilon = 0.34 # You may need to adjust this parameter
min_samples = 3  # You may need to adjust this parameter
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
#dbscan.fit(X_scaled)
dbscan.fit(X_scaled)


DBSCAN(eps=0.34, min_samples=3)

In [10]:
# Extract outlier labels
outlier_mask = dbscan.labels_ == -1

# Get the outliers
outliers = X[outlier_mask]

print(len(outliers))

1211


In [11]:
len(set(dbscan.labels_))

9

In [12]:
# Analyze outliers
print(len(df[outlier_mask]))


1211


In [21]:
non_outlier_mask=dbscan.labels_ != -1

In [23]:
print(len(df[non_outlier_mask]))

99033


In [15]:
def display_df(df: pd.DataFrame, title: str = ""):
  display(df.style.hide(axis="index").set_caption(title))

In [16]:
TGT = "readmitted"

In [17]:
prop = df.loc[:, TGT].value_counts().reset_index()
display_df(prop.sort_values("readmitted", ascending = False), "full dataset")

prop = df.loc[list(outlier_mask), TGT].value_counts().reset_index()
display_df(prop.sort_values("readmitted", ascending = False), "DBscan outliers")

index,readmitted
NO,53821
>30,35173
<30,11250


index,readmitted
NO,629
>30,427
<30,155


In [18]:
stats.chi2_contingency(np.array([[54864, 35545 , 11357], [629, 427 , 155]]))

Chi2ContingencyResult(statistic=3.778073768423819, pvalue=0.15121737856959827, dof=2, expected_freq=array([[54840.40745021, 35548.97260553, 11376.61994426],
       [  652.59254979,   423.02739447,   135.38005574]]))

In [24]:
output_df=df[non_outlier_mask]

In [25]:
output_df.to_csv("diabetic_data_cleaned_pca_som_ae_dbscan.csv")
output_df.to_parquet("diabetic_data_cleaned_pca_som_ae_dbscan.parquet")